Implement a model per day. Log:
* 6-15 Adaboost

In [104]:
import numpy as np 

class DecisionStump:
    def __init_(self):
        self.polarity = 1
        self.feature_idx = None
        self.threshold = None
        self.weight = None # estimator weight, not sample weight

    def predict(self, X):
        n_samples = X.shape[0]
        X_column = X[:, self.feature_idx]
        y_preds = np.ones(n_samples)
        if self.polarity == 1:
            y_preds[X_column <= self.threshold] = -1
        else:
            y_preds[X_column > self.threshold] = -1
        return y_preds

    
class AdaBoost:
    def __init__(self, n_estimators=5):
        self.n_estimators = n_estimators
    
    def _boost(self, X, y, sample_weight):
        '''Implement a single boost using the SAMME discrete algorithm.'''
        estimator = DecisionStump()
        min_error = 0x3f3f3f3f
        n_samples, n_features = X.shape

        for idx in range(n_features):
            X_column = X[:, idx]
            for t in np.unique(X_column):
                preds = np.ones(n_samples)
                preds[X_column <= t] = -1

                '''Use np.sum instead of sum speeds up calculation and improve perfromance'''
                error = np.sum(sample_weight[preds != y])

                polarity = 1 if error <= 0.5 else -1
                error = min(error, 1 - error)

                if error < min_error:
                    min_error = error
                    estimator.polarity = polarity
                    estimator.threshold = t 
                    estimator.feature_idx = idx 

        estimator.weight = 0.5 * np.log((1 - error) / (error + 1e-10))
        y_preds = estimator.predict(X)
        sample_weight *= np.exp(- estimator.weight * y * y_preds)
        sample_weight /= np.sum(sample_weight)

        self.estimators.append(estimator)
        return sample_weight
        
        
    def fit(self, X, y):
        '''Build a boosted classifier from the training set (X, y)'''
        n_samples, n_features = X.shape
        sample_weight = np.full(n_samples, 1 / n_samples)
        self.estimators = []
        
        for _ in range(self.n_estimators): # update sample_weight at each round
            sample_weight = self._boost(X, y, sample_weight)

        return self
    
    def predict(self, X):
        y_preds = np.sum([c.weight * c.predict(X) for c in self.estimators], axis=0)
        sign_preds = np.sign(y_preds)
        return sign_preds
    
np.full(5, 1)        

array([1, 1, 1, 1, 1])

In [105]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

d = load_breast_cancer()
X, y = d.data, d.target 
y[y==0] = -1
model = AdaBoostClassifier(n_estimators=10)

def test_accuracy_score(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, test_size=0.2)
    model.fit(X_train, y_train)
    print("Accuracy score, ", accuracy_score(y_val, model.predict(X_val)))

test_accuracy_score(model, X, y)    
%time test_accuracy_score(AdaBoost(n_estimators=10), X, y)

Accuracy score,  0.9649122807017544
Accuracy score,  0.9649122807017544
CPU times: user 3.33 s, sys: 20.4 ms, total: 3.35 s
Wall time: 3.37 s


In [91]:
y = [1, 2, 1, 2, 3]
np.sum(y)

9